In [1]:
import numpy as np
import pandas as pd

from Tkinter import Tk
from Tkinter import *
import Tkinter, Tkconstants, tkFileDialog
from tkFileDialog import askopenfilename
from tkCommonDialog import Dialog
import tkMessageBox as mb


Select the .xlsx file from which you would like to remove outliers.  

In [2]:
mb.showinfo("qPCR Processer", "Select the file you would like to filter.")
# get the file name from the user using tkinter
Tk().withdraw() # we don't want a full GUI, so keep the root window from appearing
path = askopenfilename() # show an "Open" dialog box and return the path

Create a DataFrame with values from the .xlsx file you selected above.

In [3]:
df = pd.read_excel(path, 
                   header=None, 
                   names=['RowCol',1,2,3,'replicate','Ct',6,7,8,9,10,11,12])

#### Edit() class to remove outliers:
If a Ct value in a triplicate is > 1 standard devation from the mean, you can remove it using the remove_outliers() method of the Edit class.

In [4]:
class Edit(object):
    def __init__(self, data, cutoff):
        # data is a DataFrame like dfa above
        self.data = data
        self.cutoff = cutoff
        self.trips = data.groupby('replicate')
        # define some important variables for the function loop to use
        self.std = self.trips['Ct'].std()
        self.ul = self.trips['Ct'].mean() + self.std
        self.ll = self.trips['Ct'].mean() - self.std
        
    def remove_outliers(self):
        
        for group in self.trips.groups:
                for row in range(0, len(self.trips.groups[group])):

                    index = self.trips.groups[group][row]                    

                    if (self.std[group] > self.cutoff) and (self.data['Ct'][index] > self.ul[group] or self.data['Ct'][index] < self.ll[group]):
                        #print(self.ll[group], dfa['Ct'][index], self.ul[group])
                        self.data['Ct'][index] = 'outlier'

        return self.data

Below is an instantiation of the Edit class. It is called with the dataframe created above and the minimum standard deviation that indicates a need for outlier removal. 

In [5]:
test1 = Edit(df, 0.3)
df_no_outs = test1.remove_outliers()
df_no_outs.head()

C:\Users\John Cooper\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\John Cooper\AppData\Local\Continuum\Anaconda2\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,RowCol,1,2,3,replicate,Ct,6,7,8,9,10,11,12
0,A01,SYBR,Unkn,NaN,1,24.15,23.82,0.296,0,NaN,0,0,NaN
1,A02,SYBR,Unkn,NaN,1,23.74,23.82,0.296,0,NaN,0,0,NaN
2,A03,SYBR,Unkn,NaN,1,23.58,23.82,0.296,0,NaN,0,0,NaN
3,A04,SYBR,Unkn,NaN,2,23.66,23.65,0.019,0,NaN,0,0,NaN
4,A05,SYBR,Unkn,NaN,2,23.63,23.65,0.019,0,NaN,0,0,NaN


In [6]:
# Tell the user to select processed data output directory
mb.showinfo("qPCR Processer",
"Choose your output file. Include the file extension .xlsx in the title your write")

# ask the user to 'save as' the output file
root = Tk()
root.filename = tkFileDialog.asksaveasfilename(
    initialdir = str(path),
    title = "Select file",
    filetypes = (("xlsx files","*.xlsx"),("all files","*.*"))
    )

output_path = str(root.filename) 

writer = pd.ExcelWriter(output_path)
df.to_excel(writer, 'Sheet1', header=None)
writer.save()


# quit python. I'm not sure how else to end tkinter get rid of the annoying 
# tkinter windows. This ends the whole kernel too so you if you want to do another
# sheet you need to rerun the whole kernel from the top.
quit()